In [1]:
import numpy as np
import os
import re
import pandas as pd
import geopandas as gpd
from datetime import datetime, timedelta
from tqdm import tqdm
tqdm.pandas()
import time
from hlsstack.hls_funcs.smooth import despike_ts, double_savgol
import gc
from glob import glob
from itertools import chain
import matplotlib.pyplot as plt
import seaborn as sns

ERROR 1: PROJ: proj_create_from_database: Open of /project/cper_neon_aop/conda_envs/hls_nrt_env/share/proj failed


In [2]:
import sys
sys.path.insert(1, '/project/cper_neon_aop/hls_nrt/params')
from ltar_jer_params_smooth import *

In [42]:
from importlib import reload
import sys
reload(sys.modules["ltar_jer_params_smooth"])
from ltar_jer_params_smooth import *

In [43]:
def adapt_smooth(ts, dates, despike=True, dat_thresh=None):
    ct_valid = sum(~np.isnan(ts))
    ct_total = len(ts)
    avg_gap = ct_total/ct_valid
    if ct_valid > 0:
        if avg_gap > 15:
            despike = False
        if despike:
            if dat_thresh is None:
                _dat_thresh = np.ptp(ts.values) * 0.10
            else:
                _dat_thresh = dat_thresh
            ts_ds = despike_ts(ts.values, dat_thresh=_dat_thresh, days_thresh=45)
        else:
            ts_ds = ts.values
        if avg_gap > 10:
            ts_smooth = double_savgol(ts_ds, double=True, window1_max=7, window2=31, limit=91)
        elif avg_gap > 7:
             ts_smooth = double_savgol(ts_ds, double=True, window1_max=5, window2=41, limit=91)
        elif avg_gap > 5:
             ts_smooth = double_savgol(ts_ds, double=True, window1_max=5, window2=51, limit=91)
        else:
            ts_smooth = double_savgol(ts_ds, double=False, window2=51, limit=91)
    else:
        ts_smooth = ts
    return pd.Series(ts_smooth, ts.index.get_level_values(-1))

In [45]:
fLIST = glob(inPATH_wc)
try:
    df = pd.read_csv(inPATH_dat, parse_dates=[date_col])
except ValueError:
    df = pd.read_csv(inPATH_dat)

In [47]:
if preprocess is not None:
    df = preprocess(df)

In [48]:
df.head(5)

,year,site,zone,season,sampling_start,sampling_end,sampling_totalDays,quadrats,A_FORB,A_GRASS,...,P_S-SHR,P_SHRUB,P_ST-SU,UNK_FORB,UNK_GRASS,YUEL,pasture,UnitID,herb_kg_ha,Date
134,2013,CDRRC Pasture 3,E,FALL,2013-10-22,2013-10-22,1,32,25.057980,8.058054,...,0.812518,47.649888,0.0,0.005467,0.0,0.000000,CP3N,CP3NE,660.901083,2013-10-22
135,2013,CDRRC Pasture 3,E,SPRING,2013-04-25,2013-04-25,1,32,0.000000,0.000000,...,0.000000,16.835537,0.0,0.000000,0.0,0.000000,CP3N,CP3NE,29.049387,2013-04-25
136,2013,CDRRC Pasture 3,G,FALL,2013-10-23,2013-10-23,1,32,47.745999,5.285760,...,0.000000,3.149081,0.0,0.006212,0.0,48.611075,CP3N,CP3NG,824.938208,2013-10-23
137,2013,CDRRC Pasture 3,G,SPRING,2013-04-25,2013-04-25,1,32,0.003655,0.000000,...,0.000000,2.077505,0.0,0.000000,0.0,24.243222,CP3N,CP3NG,29.520530,2013-04-25
138,2013,CDRRC Pasture 3,S,FALL,2013-10-25,2013-10-25,1,32,8.675120,6.869720,...,1.176315,43.818227,0.0,0.650871,0.0,3.158547,CP3N,CP3NS,315.524120,2013-10-25


In [49]:
df_out = None

In [50]:
for yr in df[date_col].dt.year.unique():
    print(yr)

    # get list of files that match year based on path
    fLIST_yr = [x for x in fLIST if str(yr) in os.path.basename(x).split('_')]

    # combine all individual tile files into one dataframe
    df_yr = pd.concat([pd.read_csv(x, parse_dates=[1]) for x in fLIST_yr])
    
    # rename columns to match ground data
    df_yr = df_yr.rename(columns={'time': date_col,
                                  'Plot': id_col})
    
    # convert date to datetime
    df_yr[date_col] = pd.to_datetime(df_yr[date_col])
    
    # remove all non-plot data
    #df_yr = df_yr[df_yr[id_col] != 'UNK']
    
    # remove any dates where bands are negative
    df_yr = df_yr[~((df_yr[band_list] < 0).any(axis=1))]

    # drop any duplicates by taking mean for each plot and date (might exist e.g., from plot on edge of tile during extraction)
    df_yr = df_yr.groupby([id_col, date_col]).mean().reset_index()
    
    # get missing dates for gap-filling
    dates_missing = [x for x in pd.date_range(df_yr[date_col].min(), df_yr[date_col].max()).date if x not in df_yr[date_col].dt.date.unique()]
    
    # convert missing dates to a dataframe for combining with veg index data
    df_missing = pd.DataFrame({
        id_col: list(chain.from_iterable([list(np.repeat(x, len(dates_missing))) for x in df_yr[id_col].unique()])),
        date_col: list(chain.from_iterable(dates_missing for x in df_yr[id_col].unique()))})
    
    df_missing[date_col] = pd.to_datetime(df_missing[date_col])
    
    # combine into one dataframe for gapfilling
    df_yr_ts = pd.concat([df_yr, df_missing]).sort_values([id_col, date_col])
    
    # apply Bolton mask to extracted values
    if mask_bolton_by_id:
        ps_bolton = df_yr_ts.groupby(id_col).progress_apply(lambda x: bolton_mask_np(x['BLUE'], x['SWIR2']))
        df_yr_ts.loc[df_yr_ts[ps_bolton.droplevel(0) == 1.0].index, list(veg_dict.keys()) + band_list] = np.nan
    
    # smooth all vegetation indices to gapfill
    print('Smoothing vegetation indices')
    for vegidx in tqdm(veg_list):
        #df_yr_ts[vegidx + '_smooth'] = df_yr_ts.groupby(id_col)[vegidx].transform(lambda x: double_savgol(x.values))
        vals_smooth_yr = df_yr_ts.groupby(id_col).apply(lambda x: adapt_smooth(x[vegidx], pd.to_datetime(x[date_col])))
        if vals_smooth_yr.index.nlevels > 1:
            df_yr_ts[vegidx + '_smooth'] = vals_smooth_yr.droplevel(list(np.arange(vals_smooth_yr.index.nlevels-1)))
        else:
            df_yr_ts[vegidx + '_smooth'] = vals_smooth_yr.values.squeeze()
    print('Smoothing individual bands')
    for band in tqdm(band_list):
        #df_yr_ts[band + '_smooth'] = df_yr_ts.groupby(id_col)[band].transform(lambda x: double_savgol(x.values))
        vals_smooth_yr = df_yr_ts.groupby(id_col).apply(lambda x: adapt_smooth(x[band], pd.to_datetime(x[date_col])))
        if vals_smooth_yr.index.nlevels > 1:
            df_yr_ts[band + '_smooth'] = vals_smooth_yr.droplevel(list(np.arange(vals_smooth_yr.index.nlevels-1)))
        else:
            df_yr_ts[band + '_smooth'] = vals_smooth_yr.values.squeeze()
    
    # rename smoothed columns and drop originals
    df_yr_ts = df_yr_ts.drop(columns=veg_list + band_list)
    col_rename_dict = {c: re.sub('_smooth', '', c) for c in df_yr_ts.columns if '_smooth' in c}
    df_yr_ts = df_yr_ts.rename(columns=col_rename_dict)

    # merge with the ground data
    df_out_yr = pd.merge(df[df[date_col].dt.year == yr], 
                         df_yr_ts[[id_col, date_col] + veg_list + band_list], 
                         on=[id_col, date_col],
                         how='left')

    # create or append to final output
    if df_out is not None:
        # merge with existing ouptput dataset
        df_out = pd.concat([df_out, df_out_yr])
    else:
        # create output dataset
        df_out = df_out_yr.copy()

2013
Smoothing vegetation indices


100%|██████████| 22/22 [00:00<00:00, 28.49it/s]


Smoothing individual bands


100%|██████████| 6/6 [00:00<00:00, 59.47it/s]


2014
Smoothing vegetation indices


...

100%|██████████| 6/6 [00:00<00:00, 16.11it/s]


2023
Smoothing vegetation indices


100%|██████████| 22/22 [00:01<00:00, 15.07it/s]


Smoothing individual bands


100%|██████████| 6/6 [00:00<00:00, 15.63it/s]


In [51]:
df_out.head(5)

,year,site,zone,season,sampling_start,sampling_end,sampling_totalDays,quadrats,A_FORB,A_GRASS,...,BAI_146,BAI_236,BAI_246,BAI_346,BLUE,GREEN,RED,NIR1,SWIR1,SWIR2
0,2013,CDRRC Pasture 3,E,FALL,2013-10-22,2013-10-22,1,32,25.057980,8.058054,...,113.459349,130.637941,112.802635,112.088570,759.362976,1247.499091,1851.627414,3392.897164,3430.151494,2739.368090
1,2013,CDRRC Pasture 3,E,SPRING,2013-04-25,2013-04-25,1,32,0.000000,0.000000,...,134.064732,129.139610,136.155268,146.374845,993.505859,1581.433594,2423.125000,3199.978516,4248.328125,4054.548828
2,2013,CDRRC Pasture 3,G,FALL,2013-10-23,2013-10-23,1,32,47.745999,5.285760,...,111.787913,132.514029,110.941515,109.400772,763.554355,1227.710215,1764.185706,3345.240564,3288.044170,2476.585911
3,2013,CDRRC Pasture 3,G,SPRING,2013-04-25,2013-04-25,1,32,0.003655,0.000000,...,133.015548,129.008011,135.596815,145.480348,1135.212121,1764.219697,2597.839015,3384.662879,4398.956439,4191.200758
4,2013,CDRRC Pasture 3,S,FALL,2013-10-25,2013-10-25,1,32,8.675120,6.869720,...,120.515267,129.127841,120.770896,122.510752,814.471282,1334.083181,2002.945248,3229.836366,3541.966432,3037.093667


In [52]:
# look for any missing data
df_out[df_out['NDVI'].isnull()].sort_values(id_col)

,year,site,zone,season,sampling_start,sampling_end,sampling_totalDays,quadrats,A_FORB,A_GRASS,...,BAI_146,BAI_236,BAI_246,BAI_346,BLUE,GREEN,RED,NIR1,SWIR1,SWIR2
2,2015,CDRRC Pasture 3,G,FALL,2015-10-07,2015-10-08,2,32,12.119098,0.294313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,CDRRC Pasture 3,G,SPRING,2015-04-22,2015-04-23,2,32,12.985366,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# look for any duplicated ID's
df_out[df_out.duplicated(subset=[id_col, date_col], keep=False)].sort_values(id_col)

,year,site,zone,season,sampling_start,sampling_end,sampling_totalDays,quadrats,A_FORB,A_GRASS,...,BAI_146,BAI_236,BAI_246,BAI_346,BLUE,GREEN,RED,NIR1,SWIR1,SWIR2


In [54]:
# look for any duplicated ID's
df_out[df_out.duplicated(subset=[id_col, date_col, 'NDVI'], keep=False)].sort_values(id_col)

,year,site,zone,season,sampling_start,sampling_end,sampling_totalDays,quadrats,A_FORB,A_GRASS,...,BAI_146,BAI_236,BAI_246,BAI_346,BLUE,GREEN,RED,NIR1,SWIR1,SWIR2


In [55]:
# get mean or drop duplicated ID's
#df_out = df_out.drop_duplicates(subset=[id_col, date_col], keep='first')

In [56]:
df_out.to_csv(outPATH, index=False)